# Training

In [23]:
import pandas as pd
import numpy as np
import os
import random

# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import importlib
import train_models_util
import feature_importance
importlib.reload(train_models_util)
importlib.reload(feature_importance)
from train_models_util import (
    train_xgb_optuna, 
    train_catboost_optuna, 
    train_lgbm_optuna,
    train_best_base_models_from_mlflow,
    train_ensemble, 
    train_neural_network_ensemble,
    train_tree_optuna
)
from evaluate_models_util import evaluate_and_log
from feature_importance import select_important_features, get_top_features_shap

# set all random seeds for reproducibility
np.random.seed(42)
random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)

In [24]:
EXPERIMENT_NAME = "Fraud-detection-custom-thresholds"

In [25]:
data_folder = 'ieee-fraud-detection-data/processed/'

train = pd.read_csv(f'{data_folder}train_processed.csv')
test = pd.read_csv(f'{data_folder}test_processed.csv')
target = pd.read_csv(f'{data_folder}target.csv')['isFraud']

In [26]:
cat_cols = np.array(train.select_dtypes(include=['object']).columns)
for c in cat_cols:
    if c in train.columns:
        if train[c].isnull().sum()>0 or test[c].isnull().sum()>0:
            train[c].fillna('missing', inplace=True)
            test[c].fillna('missing', inplace=True)
        train[c] = train[c].astype('category')
        test[c] = test[c].astype('category')

## 1. Full Features

### 1.1. XGBoost

In [27]:
model_xgboost_full, best_params_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, hist_df, plot_paths = train_xgb_optuna(train, target, early_stopping_rounds=10)

[I 2025-11-26 13:59:48,654] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-26 14:05:07,136] Trial 0 finished with value: 0.47088881586875225 and parameters: {'weight_factor': 134.27572563618787, 'learning_rate': 0.13729089560689473, 'max_depth': 12, 'subsample': 0.7664939011352346, 'colsample_bytree': 0.719252789684635, 'min_child_weight': 0.4786916472732633, 'gamma': 3.246482780832669}. Best is trial 0 with value: 0.47088881586875225.
[I 2025-11-26 14:07:46,013] Trial 1 finished with value: 0.7973317440643434 and parameters: {'weight_factor': 496.93940924966176, 'learning_rate': 0.29158563729827763, 'max_depth': 8, 'subsample': 0.8013854295818089, 'colsample_bytree': 0.8917430171813643, 'min_child_weight': 0.07910295187645104, 'gamma': 4.15079541258868}. Best is trial 0 with value: 0.47088881586875225.
[I 2025-11-26 14:08:43,962] Trial 2 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 823.2615254473444, 'learning_rate': 0.013734775727280303, 'max_depth': 14, 'subsample': 0.5031561112692045, 'colsample_bytree': 0.672801378

[INFO] Best XGBoost params: {'weight_factor': 431.8579527448997, 'learning_rate': 0.044235227718251516, 'max_depth': 15, 'subsample': 0.9949272513380653, 'colsample_bytree': 0.5997900689812228, 'min_child_weight': 0.0028875632302169415, 'gamma': 0.7944884587778276}


In [28]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.6649371280552258), 'pr_auc': np.float64(0.051895584205906765), 'precision': 0.0349933958749619, 'recall': 1.0, 'f1': 0.06762052012009064, 'custom_loss': np.float64(0.9650066041250381)}
[INFO] Logged SHAP summary plot.


2025/11/26 18:07:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.6649371280552258),
 'pr_auc': np.float64(0.051895584205906765),
 'precision': 0.0349933958749619,
 'recall': 1.0,
 'f1': 0.06762052012009064,
 'custom_loss': np.float64(0.9650066041250381)}

In [29]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.6649371280552258), 'pr_auc': np.float64(0.051895584205906765), 'precision': 0.0349933958749619, 'recall': 1.0, 'f1': 0.06762052012009064, 'custom_loss': np.float64(0.9650066041250381)}
[INFO] Logged SHAP summary plot.


2025/11/26 18:07:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.6649371280552258),
 'pr_auc': np.float64(0.051895584205906765),
 'precision': 0.0349933958749619,
 'recall': 1.0,
 'f1': 0.06762052012009064,
 'custom_loss': np.float64(0.9650066041250381)}

In [30]:
evaluate_and_log(model_xgboost_full, X_va_xgboost_full, y_va_xgboost_full, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.6649371280552258), 'pr_auc': np.float64(0.051895584205906765), 'precision': 0.0349933958749619, 'recall': 1.0, 'f1': 0.06762052012009064, 'custom_loss': np.float64(0.9650066041250381)}
[INFO] Logged SHAP summary plot.


2025/11/26 18:07:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.6649371280552258),
 'pr_auc': np.float64(0.051895584205906765),
 'precision': 0.0349933958749619,
 'recall': 1.0,
 'f1': 0.06762052012009064,
 'custom_loss': np.float64(0.9650066041250381)}

### 1.2. LightGBM

In [31]:
model_lgbm_full, best_params_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, hist_df, plot_paths = train_lgbm_optuna(train, target)

[I 2025-11-26 18:07:35,693] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-26 18:08:37,381] Trial 0 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 799.8887477754231, 'num_leaves': 72, 'max_depth': 19, 'learning_rate': 0.029454087434079373, 'feature_fraction': 0.7277078554305384, 'bagging_fraction': 0.7287395895433799, 'bagging_freq': 4, 'min_child_samples': 26, 'lambda_l1': 2.459479077273741e-07, 'lambda_l2': 4.6310415923707434e-08}. Best is trial 0 with value: 3.4989162462416536.
[I 2025-11-26 18:09:46,731] Trial 1 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 354.28503545311355, 'num_leaves': 216, 'max_depth': 10, 'learning_rate': 0.032971648186683036, 'feature_fraction': 0.7399157217942542, 'bagging_fraction': 0.7279031960093602, 'bagging_freq': 9, 'min_child_samples': 92, 'lambda_l1': 0.0003515177272996902, 'lambda_l2': 0.002144546703076014}. Best is trial 0 with value: 3.4989162462416536.
[I 2025-11-26 18:11:12,183] Trial 2 finished with value: 3.4989162462416536 and parameters: {'weight_f

[INFO] Best LightGBM params: {'weight_factor': 101.49429659893593, 'num_leaves': 252, 'max_depth': 16, 'learning_rate': 0.14786343065875304, 'feature_fraction': 0.6036658478336019, 'bagging_fraction': 0.9976494567167882, 'bagging_freq': 1, 'min_child_samples': 75, 'lambda_l1': 0.002791955932040801, 'lambda_l2': 1.7096539255755277e-06}


In [32]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9750039758847451), 'pr_auc': np.float64(0.9188383936008578), 'precision': 0.9402625234395928, 'recall': 0.8492620372610694, 'f1': 0.8924485125858124, 'custom_loss': np.float64(0.5293714227656043)}
[INFO] Logged SHAP summary plot.


2025/11/26 19:07:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9750039758847451),
 'pr_auc': np.float64(0.9188383936008578),
 'precision': 0.9402625234395928,
 'recall': 0.8492620372610694,
 'f1': 0.8924485125858124,
 'custom_loss': np.float64(0.5293714227656043)}

In [33]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9750039758847451), 'pr_auc': np.float64(0.9188383936008578), 'precision': 0.9253886010362694, 'recall': 0.8642632470360513, 'f1': 0.8937820593018891, 'custom_loss': np.float64(0.4774274392928506)}
[INFO] Logged SHAP summary plot.


2025/11/26 19:07:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9750039758847451),
 'pr_auc': np.float64(0.9188383936008578),
 'precision': 0.9253886010362694,
 'recall': 0.8642632470360513,
 'f1': 0.8937820593018891,
 'custom_loss': np.float64(0.4774274392928506)}

In [34]:
evaluate_and_log(model_lgbm_full, X_va_lgbm_full, y_va_lgbm_full, experiment_name=EXPERIMENT_NAME, run_name="LightGBM_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.10)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9750039758847451), 'pr_auc': np.float64(0.9188383936008578), 'precision': 0.9021467798302546, 'recall': 0.8744253568836197, 'f1': 0.8880697874431749, 'custom_loss': np.float64(0.4427473160158499)}
[INFO] Logged SHAP summary plot.


2025/11/26 19:08:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9750039758847451),
 'pr_auc': np.float64(0.9188383936008578),
 'precision': 0.9021467798302546,
 'recall': 0.8744253568836197,
 'f1': 0.8880697874431749,
 'custom_loss': np.float64(0.4427473160158499)}

### 1.3. CatBoost

In [ ]:
model_catboost_full, best_params_catboost_full, X_va_catboost_full, y_va_catboost_full, hist_df, plot_paths = train_catboost_optuna(train, target)

[I 2025-11-26 19:08:53,938] A new study created in memory with name: catboost_custom_cost_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-26 19:18:45,763] Trial 0 finished with value: 0.964964269859304 and parameters: {'weight_factor': 207.40376845147583, 'learning_rate': 0.043444901231957996, 'depth': 10, 'l2_leaf_reg': 8.190413281865796, 'subsample': 0.7937582767415685, 'border_count': 91}. Best is trial 0 with value: 0.964964269859304.
[I 2025-11-26 19:30:24,655] Trial 1 finished with value: 0.9650044874576784 and parameters: {'weight_factor': 531.5375594885923, 'learning_rate': 0.012600629275413316, 'depth': 12, 'l2_leaf_reg': 9.203161827882193, 'subsample': 0.8520183053142552, 'border_count': 35}. Best is trial 0 with value: 0.964964269859304.
[I 2025-11-26 19:48:54,206] Trial 2 finished with value: 0.9648012828845157 and parameters: {'weight_factor': 839.8223278389047, 'learning_rate': 0.022406941784403004, 'depth': 14, 'l2_leaf_reg': 6.646705874770338, 'subsample': 0.9721685621578435, 'border_count': 106}. Best is trial 2 with value: 0.9648012828845157.
[I 2025-11-26 20:06:21,511] Trial 3 finished with 

In [ ]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.9642747151216507, 'recall': 0.7575610936365836, 'f1': 0.8485094850948509, 'custom_loss': np.float64(0.8493582145155282)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:09:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.9642747151216507,
 'recall': 0.7575610936365836,
 'f1': 0.8485094850948509,
 'custom_loss': np.float64(0.8493582145155282)}

In [ ]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.9323697845650395, 'recall': 0.827244132591338, 'f1': 0.8766666666666667, 'custom_loss': np.float64(0.6066312188844108)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:26:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.9323697845650395,
 'recall': 0.827244132591338,
 'f1': 0.8766666666666667,
 'custom_loss': np.float64(0.6066312188844108)}

In [ ]:
evaluate_and_log(model_catboost_full, X_va_catboost_full, y_va_catboost_full, experiment_name=EXPERIMENT_NAME, run_name="CATboost_Optuna_fullfeatures", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.1)

[INFO] Logged metrics: {'roc_auc': np.float64(0.9792185548859704), 'pr_auc': np.float64(0.9085207718513296), 'precision': 0.8489191729323309, 'recall': 0.874183401887249, 'f1': 0.8613660746215281, 'custom_loss': np.float64(0.44571917228299524)}
[INFO] Logged SHAP summary plot.


2025/11/26 12:44:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9792185548859704),
 'pr_auc': np.float64(0.9085207718513296),
 'precision': 0.8489191729323309,
 'recall': 0.874183401887249,
 'f1': 0.8613660746215281,
 'custom_loss': np.float64(0.44571917228299524)}

## 2. Reduced Features

In [ ]:
# ---- 1. Compute SHAP importance for each model ----
df_xgb = get_top_features_shap(model_xgboost_full, train, target).rename(
    columns={"importance": "importance_xgb"}
)
df_lgbm = get_top_features_shap(model_lgbm_full, train, target).rename(
    columns={"importance": "importance_lgbm"}
)
df_cat = get_top_features_shap(model_catboost_full, train, target).rename(
    columns={"importance": "importance_cat"}
)

# ---- 2. Merge all importance tables ----
df = (
    df_xgb.merge(df_lgbm, on="feature")
          .merge(df_cat, on="feature")
)

# ---- 3. Rank features within each model (1 = most important) ----
df["rank_xgb"] = df["importance_xgb"].rank(method="min", ascending=False)
df["rank_lgbm"] = df["importance_lgbm"].rank(method="min", ascending=False)
df["rank_cat"] = df["importance_cat"].rank(method="min", ascending=False)

# ---- 4. Determine the top 30% threshold ----
n_features = len(df)
top_30_cutoff = int(n_features * 0.30)

# ---- 5. Check if a feature appears in top 30% in each model ----
df["in_top_xgb"]  = df["rank_xgb"]  <= top_30_cutoff
df["in_top_lgbm"] = df["rank_lgbm"] <= top_30_cutoff
df["in_top_cat"]  = df["rank_cat"]  <= top_30_cutoff

# ---- 6. Apply the rule: “Top 30% for at least 2 models” ----
df["top_count"] = (
    df["in_top_xgb"].astype(int)
  + df["in_top_lgbm"].astype(int)
  + df["in_top_cat"].astype(int)
)

df_selected = df[df["top_count"] >= 2]  # final selected features

# ---- 7. Sort by mean importance (optional but nice) ----
df["importance_mean"] = df[["importance_xgb","importance_lgbm","importance_cat"]].mean(axis=1)
df_selected = df_selected.sort_values("importance_mean", ascending=False)

# ---- 8. Save to CSV ----
df.to_csv(f"{data_folder}/feature_importances_shap_fullmodels.csv", index=False)
df_selected.to_csv(f"{data_folder}/selected_features_shap.csv", index=False)

# ---- 9. List of selected feature names ----
selected_feature_list = df_selected["feature"].tolist()


In [ ]:
X_reduced = train[selected_feature_list]

In [ ]:
del df_xgb, df_lgbm, df_cat, df_selected, df

In [ ]:
# X_reduced, feature_ranking = select_important_features(
#     train,
#     target,
#     top_n=100,                # or None to use cumulative 95%
#     use_shap=True,            # optional, slower but more accurate
#     remove_correlated=True,
#     correlation_threshold=0.95
# )

[INFO] Training LightGBM for feature importance estimation...
[INFO] Computing SHAP values for refined importance...
[INFO] Keeping top 100 features by importance.
[INFO] Performing correlation pruning (threshold=0.95)...
[INFO] Dropping 15 correlated features.
[DONE] Selected 85 features out of 407.


### 2.1. XGBoost 

In [47]:
model_xgboost_reduced, best_params_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, hist_df, plot_paths = train_xgb_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_xgboost_reduced, X_va_xgboost_reduced, y_va_xgboost_reduced, experiment_name=EXPERIMENT_NAME, run_name="XGBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 02:07:42,982] A new study created in memory with name: xgboost_customloss_optimization


[Optuna XGBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 02:10:21,382] Trial 0 finished with value: 0.47448512967185286 and parameters: {'weight_factor': 303.03445169252836, 'learning_rate': 0.10083183895081857, 'max_depth': 11, 'subsample': 0.8963608060733238, 'colsample_bytree': 0.6785103891501436, 'min_child_weight': 0.07640426499273617, 'gamma': 4.238768344859967}. Best is trial 0 with value: 0.47448512967185286.
[I 2025-11-20 02:15:17,159] Trial 1 finished with value: 0.47430942625086236 and parameters: {'weight_factor': 384.04306259406184, 'learning_rate': 0.043534070489637824, 'max_depth': 14, 'subsample': 0.7174468047365997, 'colsample_bytree': 0.7170701546516388, 'min_child_weight': 0.45039260296668343, 'gamma': 0.9937966768189904}. Best is trial 1 with value: 0.47430942625086236.
[I 2025-11-20 02:21:49,082] Trial 2 finished with value: 0.438930817423392 and parameters: {'weight_factor': 384.6739597646329, 'learning_rate': 0.029851398336222162, 'max_depth': 11, 'subsample': 0.9780693528395272, 'colsample_bytree': 0.665

[INFO] Best XGBoost params: {'weight_factor': 563.1612148501741, 'learning_rate': 0.016063616494696824, 'max_depth': 15, 'subsample': 0.9610875639100243, 'colsample_bytree': 0.571910755843421, 'min_child_weight': 0.0054253161383116325, 'gamma': 3.032335195843312}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.8718726307808946, 'recall': 0.8347447374788289, 'f1': 0.8529048207663782, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:15:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9783492682732146), 'pr_auc': np.float64(0.8923424129990234), 'precision': 0.7010428736964078, 'recall': 0.8782966368255505, 'f1': 0.7797229083879282, 'custom_loss': np.float64(0.5825769634571748)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:16:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9783492682732146),
 'pr_auc': np.float64(0.8923424129990234),
 'precision': 0.7010428736964078,
 'recall': 0.8782966368255505,
 'f1': 0.7797229083879282,
 'custom_loss': np.float64(0.5825769634571748)}

### 2.2. LightGBM

In [48]:
model_lgmb_reduced, best_params_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, hist_df, plot_paths = train_lgbm_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_lgmb_reduced, X_va_lgmb_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="LGBM_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:16:45,506] A new study created in memory with name: lgbm_custom_cost_optimization


[Optuna LightGBM Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:17:31,240] Trial 0 finished with value: 0.6125331219595925 and parameters: {'weight_factor': 988.4441409271524, 'num_leaves': 166, 'max_depth': 20, 'learning_rate': 0.18528910883777497, 'feature_fraction': 0.9538139183876564, 'bagging_fraction': 0.7829489483618187, 'bagging_freq': 5, 'min_child_samples': 65, 'lambda_l1': 3.14241707785604e-05, 'lambda_l2': 0.011144987897729686}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:17:50,230] Trial 1 finished with value: 3.4989162462416536 and parameters: {'weight_factor': 763.9845955234861, 'num_leaves': 115, 'max_depth': 19, 'learning_rate': 0.03601466570668402, 'feature_fraction': 0.9765467245612626, 'bagging_fraction': 0.6274827760378981, 'bagging_freq': 8, 'min_child_samples': 58, 'lambda_l1': 1.4364084634718396e-07, 'lambda_l2': 0.10126313290615058}. Best is trial 0 with value: 0.6125331219595925.
[I 2025-11-20 05:18:13,468] Trial 2 finished with value: 0.8158042208441103 and parameters: {'weight_factor

[INFO] Best LightGBM params: {'weight_factor': 595.0221512419944, 'num_leaves': 76, 'max_depth': 20, 'learning_rate': 0.19236941161792298, 'feature_fraction': 0.879964550074313, 'bagging_fraction': 0.9215867301351228, 'bagging_freq': 4, 'min_child_samples': 84, 'lambda_l1': 8.965510102130184, 'lambda_l2': 1.257085921083824e-06}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.8477306002928258, 'recall': 0.8405516573917251, 'f1': 0.8441258656299356, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:31:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9737186890783829), 'pr_auc': np.float64(0.8897176420223434), 'precision': 0.7289436477479297, 'recall': 0.8732155819017663, 'f1': 0.7945838837516512, 'custom_loss': np.float64(0.5632471974802723)}
[INFO] Logged SHAP summary plot.


2025/11/20 05:32:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9737186890783829),
 'pr_auc': np.float64(0.8897176420223434),
 'precision': 0.7289436477479297,
 'recall': 0.8732155819017663,
 'f1': 0.7945838837516512,
 'custom_loss': np.float64(0.5632471974802723)}

### 2.3. CatBoost

In [49]:
model_catoost_reduced, best_params_catoost_reduced, X_va_catoost_reduced, y_va_catoost_reduced, hist_df, plot_paths = train_catboost_optuna(X_reduced, target, n_trials=30)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths)
evaluate_and_log(model_catoost_reduced, X_va_catoost_reduced, y_va_lgmb_reduced, experiment_name=EXPERIMENT_NAME, run_name="CATBoost_Optuna_Reduced", hp_search_history=hist_df, hp_search_plots= plot_paths, prediction_threshold=0.25)

[I 2025-11-20 05:32:09,180] A new study created in memory with name: catboost_custom_cost_optimization


[Optuna CatBoost Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-20 05:35:30,115] Trial 0 finished with value: 0.9650108375375834 and parameters: {'weight_factor': 163.7397266034792, 'learning_rate': 0.03120659802284773, 'depth': 12, 'l2_leaf_reg': 8.181175589517238, 'subsample': 0.6736681243794981, 'border_count': 107}. Best is trial 0 with value: 0.9650108375375834.
[I 2025-11-20 05:42:17,422] Trial 1 finished with value: 0.9649769704447559 and parameters: {'weight_factor': 974.137254461418, 'learning_rate': 0.24457363544379654, 'depth': 16, 'l2_leaf_reg': 4.954780337206101, 'subsample': 0.874814049584014, 'border_count': 54}. Best is trial 1 with value: 0.9649769704447559.
[I 2025-11-20 05:49:22,039] Trial 2 finished with value: 0.9649515700355265 and parameters: {'weight_factor': 829.7542285570263, 'learning_rate': 0.1173314516038542, 'depth': 15, 'l2_leaf_reg': 5.661806434494909, 'subsample': 0.9543732606288229, 'border_count': 97}. Best is trial 2 with value: 0.9649515700355265.
[I 2025-11-20 05:55:44,572] Trial 3 finished with valu

[INFO] Best CatBoost params: {'weight_factor': 826.5125110443701, 'learning_rate': 0.05013487336385276, 'depth': 10, 'l2_leaf_reg': 2.1356969501070076, 'subsample': 0.7829974303923855, 'border_count': 139}
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.6505289582212659, 'recall': 0.8778127268328091, 'f1': 0.7472708547888774, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:45:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.
[INFO] Logged metrics: {'roc_auc': np.float64(0.9726674973133655), 'pr_auc': np.float64(0.8668056153521687), 'precision': 0.3822463768115942, 'recall': 0.9189450762158239, 'f1': 0.539910441395977, 'custom_loss': np.float64(0.44407660785044195)}
[INFO] Logged SHAP summary plot.


2025/11/20 07:48:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.9726674973133655),
 'pr_auc': np.float64(0.8668056153521687),
 'precision': 0.3822463768115942,
 'recall': 0.9189450762158239,
 'f1': 0.539910441395977,
 'custom_loss': np.float64(0.44407660785044195)}

## 3. Ensemble

In [ ]:
base_models, X_tr, X_va, y_tr, y_va, X_test, y_test, best_params_dict = train_best_base_models_from_mlflow(
    train, target, test_size=0.15 ,experiment_name=EXPERIMENT_NAME
)

[INFO] Searching for best xgboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best xgboost run 2de7afde847a4e978f4d7914a09c9200 with pr_auc=0.9136
[INFO] Retraining xgboost with best parameters...
[INFO] Searching for best catboost model in experiment 'FRAUD_DETECTION'...
[INFO] Found best catboost run 3c8041e4dcf64b9c84c2d5bdd039c60c with pr_auc=0.9082
[INFO] Retraining catboost with best parameters...
[INFO] Searching for best lightgbm model in experiment 'FRAUD_DETECTION'...
[INFO] Found best lightgbm run cd33e5f9d37a4660a9621d0bed29f886 with pr_auc=0.9174
[INFO] Retraining lightgbm with best parameters...


### 3.1. Logistic

In [ ]:
ensemble, base_models, X_meta_va, y_va, X_meta_test, y_test, best_params_log, hist_df, plot_paths = train_ensemble(
    base_models, X_tr, y_tr, X_va, y_va, X_test, y_test, n_trials=50
)

[INFO] Generating meta-features...
[INFO] xgboost validation PR-AUC: 0.8901
[INFO] catboost validation PR-AUC: 0.8128


[I 2025-11-18 03:06:15,837] A new study created in memory with name: ensemble_meta_learner_pr_auc_optimization


[INFO] lightgbm validation PR-AUC: 0.8840
[INFO] Optimizing meta-learner with Optuna...


[Optuna Ensemble Meta-Learner Tuning]:   0%|          | 0/50 [00:00<?, ?trial/s]

[I 2025-11-18 03:06:17,544] Trial 0 finished with value: 0.8968493565501713 and parameters: {'C': 0.004650653818750843, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'elasticnet', 'l1_ratio': 0.9262925490300581}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:18,422] Trial 1 finished with value: 0.8967225113225421 and parameters: {'C': 0.07273320235255908, 'class_weight': 'balanced', 'solver': 'newton-cg', 'penalty_newton-cg': 'l2'}. Best is trial 0 with value: 0.8968493565501713.
[I 2025-11-18 03:06:20,092] Trial 2 finished with value: 0.8969843620283047 and parameters: {'C': 0.05786152184681003, 'class_weight': None, 'solver': 'saga', 'penalty_saga': 'l2'}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:18,967] Trial 3 finished with value: 0.7973015325222292 and parameters: {'C': 29.13605449340287, 'class_weight': 'balanced', 'solver': 'sag', 'penalty_sag': None}. Best is trial 2 with value: 0.8969843620283047.
[I 2025-11-18 03:08:1

[INFO] Best meta-learner params: {'C': 0.21957921937861558, 'class_weight': None, 'solver': 'lbfgs', 'penalty_lbfgs': None}
[INFO] Ensemble validation PR-AUC: 0.8972
[INFO] Ensemble validation ROC-AUC: 0.9778
[INFO] Ensemble test PR-AUC: 0.8934
[INFO] Ensemble test ROC-AUC: 0.9770


In [54]:
evaluate_and_log(ensemble, pd.DataFrame(X_meta_va), y_va, experiment_name=EXPERIMENT_NAME, run_name="Ensemble_from_best_base_models", hp_search_history=hist_df)

[INFO] Logged metrics: {'roc_auc': np.float64(0.977775749485985), 'pr_auc': np.float64(0.8971915104190178), 'precision': 0.9516616314199395, 'recall': 0.7621582385676264, 'f1': 0.8464328899637243, 'custom_loss': np.float64(0.8336437836556372)}
[WARN] SHAP skipped: Model type not yet supported by TreeExplainer: <class 'sklearn.linear_model._logistic.LogisticRegression'>


2025/11/18 03:11:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[INFO] Evaluation complete and logged.


{'roc_auc': np.float64(0.977775749485985),
 'pr_auc': np.float64(0.8971915104190178),
 'precision': 0.9516616314199395,
 'recall': 0.7621582385676264,
 'f1': 0.8464328899637243,
 'custom_loss': np.float64(0.8336437836556372)}

### 3.2. NN

In [55]:
nn_model, base_mdls, X_val, y_val, X_test, y_test, params, hist, plots = train_neural_network_ensemble(
    base_models=base_models,
    X_tr=X_tr,
    y_tr=y_tr,
    X_va=X_va,
    y_va=y_va,
    X_test=X_test,
    y_test=y_test,
    n_trials=30,
    epochs=100
)

[INFO] Generating meta-features from base models...
[INFO] xgboost validation custom_loss: 0.8598
[INFO] catboost validation custom_loss: 1.3679


[I 2025-11-18 03:12:01,112] A new study created in memory with name: neural_network_ensemble_optimization


[INFO] lightgbm validation custom_loss: 0.9562
[INFO] Optimizing neural network meta-learner with Optuna...


[Optuna NN Ensemble Tuning]:   0%|          | 0/30 [00:00<?, ?trial/s]

[I 2025-11-18 03:16:57,637] Trial 0 finished with value: 0.7586107630304467 and parameters: {'n_layers': 1, 'hidden_size_0': 180, 'dropout_rate': 0.38574414781037836, 'learning_rate': 0.005570027698867698, 'weight_decay': 0.000253232244112943}. Best is trial 0 with value: 0.7586107630304467.
[I 2025-11-18 03:26:37,819] Trial 1 finished with value: 0.7185118704914146 and parameters: {'n_layers': 1, 'hidden_size_0': 38, 'dropout_rate': 0.2716836549595695, 'learning_rate': 0.0007150539132222626, 'weight_decay': 5.689583047950391e-06}. Best is trial 1 with value: 0.7185118704914146.
[I 2025-11-18 03:37:02,296] Trial 2 finished with value: 0.6539946489653538 and parameters: {'n_layers': 2, 'hidden_size_0': 200, 'hidden_size_1': 138, 'dropout_rate': 0.014344713117248387, 'learning_rate': 0.009775444648925805, 'weight_decay': 4.869795488201027e-05}. Best is trial 2 with value: 0.6539946489653538.
[I 2025-11-18 03:42:49,435] Trial 3 finished with value: 0.8218664273376909 and parameters: {'n_l

[INFO] Best neural network params: {'n_layers': 3, 'hidden_size_0': 64, 'hidden_size_1': 149, 'hidden_size_2': 123, 'dropout_rate': 0.1519020286431952, 'learning_rate': 0.006663695072321037, 'weight_decay': 1.8779452673203788e-06}
[INFO] Neural network ensemble validation custom_loss: 0.6392
[INFO] Neural network ensemble validation PR-AUC: 0.8229
[INFO] Neural network ensemble validation ROC-AUC: 0.9408
[INFO] Neural network ensemble test custom_loss: 0.6862
[INFO] Neural network ensemble test PR-AUC: 0.8160
[INFO] Neural network ensemble test ROC-AUC: 0.9344
